In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import re
import collections

# GPU varsa kullan, yoksa CPU kullan
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan Cihaz: {device}")

Kullanılan Cihaz: cuda


## VERİ

In [17]:
train_data = {
  'good': True,
  'bad': False,
  'happy': True,
  'sad': False,
  'not good': False,
  'not bad': True,
  'not happy': False,
  'not sad': True,
  'very good': True,
  'very bad': False,
  'very happy': True,
  'very sad': False,
  'i am happy': True,
  'this is good': True,
  'i am bad': False,
  'this is bad': False,
  'i am sad': False,
  'this is sad': False,
  'i am not happy': False,
  'this is not good': False,
  'i am not bad': True,
  'this is not sad': True,
  'i am very happy': True,
  'this is very good': True,
  'i am very bad': False,
  'this is very sad': False,
  'this is very happy': True,
  'i am good not bad': True,
  'this is good not bad': True,
  'i am bad not good': False,
  'i am good and happy': True,
  'this is not good and not happy': False,
  'i am not at all good': False,
  'i am not at all bad': True,
  'i am not at all happy': False,
  'this is not at all sad': True,
  'this is not at all happy': False,
  'i am good right now': True,
  'i am bad right now': False,
  'this is bad right now': False,
  'i am sad right now': False,
  'i was good earlier': True,
  'i was happy earlier': True,
  'i was bad earlier': False,
  'i was sad earlier': False,
  'i am very bad right now': False,
  'this is very good right now': True,
  'this is very sad right now': False,
  'this was bad earlier': False,
  'this was very good earlier': True,
  'this was very bad earlier': False,
  'this was very happy earlier': True,
  'this was very sad earlier': False,
  'i was good and not bad earlier': True,
  'i was not good and not happy earlier': False,
  'i am not at all bad or sad right now': True,
  'i am not at all good or happy right now': False,
  'this was not happy and not good earlier': False,
}

test_data = {
  'this is happy': True,
  'i am good': True,
  'this is not happy': False,
  'i am not good': False,
  'this is not bad': True,
  'i am not sad': True,
  'i am very good': True,
  'this is very bad': False,
  'i am very sad': False,
  'this is bad not good': False,
  'this is good and happy': True,
  'i am not good and not happy': False,
  'i am not at all sad': True,
  'this is not at all good': False,
  'this is not at all bad': True,
  'this is good right now': True,
  'this is sad right now': False,
  'this is very bad right now': False,
  'this was good earlier': True,
  'i was not happy and not good earlier': False,
}

## VERİ HAZIRLIĞI

In [18]:
def clean_and_tokenize(sentence):
    # Basit temizleme ve tokenizasyon
    sentence = re.sub(r'[^a-z ]', '', sentence.lower())
    return sentence.split()

def build_vocab(data, min_freq=1):
    # Veri setindeki tüm kelimelerden kelime haznesi oluştur
    all_tokens = []
    for sentence in data.keys():
        all_tokens.extend(clean_and_tokenize(sentence))

    token_counts = collections.Counter(all_tokens)
    # Belirli bir frekanstan az olan kelimeleri at
    vocab = [token for token, count in token_counts.items() if count >= min_freq]

    # Özel tokenlar ekle: <PAD> (doldurma) ve <UNK> (bilinmeyen)
    vocab.insert(0, '<PAD>')
    vocab.insert(1, '<UNK>')

    word_to_idx = {word: i for i, word in enumerate(vocab)}
    idx_to_word = {i: word for word, i in word_to_idx.items()}

    return word_to_idx, idx_to_word

def sentence_to_indices(sentence, word_to_idx, max_len):
    # Cümleyi temizle, tokenlara ayır ve indeks dizisine çevir
    tokens = clean_and_tokenize(sentence)
    indices = [word_to_idx.get(token, word_to_idx['<UNK>']) for token in tokens]

    # Padding veya Truncation
    if len(indices) < max_len:
        # Padding (sona <PAD> indeksi ekle)
        padding = [word_to_idx['<PAD>']] * (max_len - len(indices))
        indices.extend(padding)
    elif len(indices) > max_len:
        # Truncation (baştan kes)
        indices = indices[:max_len]

    return indices

def prepare_dataset(data, word_to_idx, max_len):
    # Veri setini PyTorch tensorlarına dönüştür
    all_indices = []
    labels = []

    for sentence, label in data.items():
        indices = sentence_to_indices(sentence, word_to_idx, max_len)
        all_indices.append(indices)
        labels.append(float(label)) # PyTorch beklentisi float

    # Numpy array'lere çevir ve sonra PyTorch tensorlarına
    # Batch boyutu ilk boyut olacak şekilde düzenle
    indices_tensor = torch.LongTensor(all_indices)
    labels_tensor = torch.FloatTensor(labels).unsqueeze(1) # BCEWithLogitsLoss için (batch_size, 1) şekli uygun olabilir

    return indices_tensor, labels_tensor

# Kelime haznesini oluştur
word_to_idx, idx_to_word = build_vocab(train_data)
vocab_size = len(word_to_idx)
print(f"Kelime Haznesi Boyutu: {vocab_size}")

# Maksimum cümle uzunluğunu belirle (padding için)
max_sequence_length = max(len(clean_and_tokenize(s)) for s in train_data.keys())
print(f"Maksimum Dizi Uzunluğu: {max_sequence_length}")

# Veri setlerini tensorlara dönüştür
train_indices_tensor, train_labels_tensor = prepare_dataset(train_data, word_to_idx, max_sequence_length)
test_indices_tensor, test_labels_tensor = prepare_dataset(test_data, word_to_idx, max_sequence_length)

print(f"Eğitim Verisi Tensor Şekli: {train_indices_tensor.shape}, Etiket Şekli: {train_labels_tensor.shape}")
print(f"Test Verisi Tensor Şekli: {test_indices_tensor.shape}, Etiket Şekli: {test_labels_tensor.shape}")

Kelime Haznesi Boyutu: 20
Maksimum Dizi Uzunluğu: 10
Eğitim Verisi Tensor Şekli: torch.Size([58, 10]), Etiket Şekli: torch.Size([58, 1])
Test Verisi Tensor Şekli: torch.Size([20, 10]), Etiket Şekli: torch.Size([20, 1])


## MODEL

In [19]:
class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
        super(SentimentRNN, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # batch_first=True: Giriş tensorunun şekli (batch, sequence, feature) olur
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size) # Tam bağlı katman (Fully Connected)

    def forward(self, text):
        # text girdisi (batch_size, sequence_length) şeklinde indeks tensoru
        embedded = self.embedding(text) # Çıktı şekli: (batch_size, sequence_length, embedding_dim)

        # RNN katmanı
        # output: Tüm zaman adımlarının gizli durumları (batch_size, sequence_length, hidden_size)
        # hidden: Son zaman adımının gizli durumu (1, batch_size, hidden_size) - SimpleRNN için
        output, hidden = self.rnn(embedded)

        # Sınıflandırma için son zaman adımının gizli durumunu kullan
        # hidden[:, -1, :] -> (batch_size, hidden_size) şeklini alır
        # SimpleRNN'de 'hidden' tensoru (num_layers * num_directions, batch_size, hidden_size) şeklindedir.
        # Bizde num_layers=1, num_directions=1. Yani hidden[0] son gizli durumdur.
        final_hidden_state = hidden[0] # Şekil: (batch_size, hidden_size)

        # Tam bağlı katmana besle
        logits = self.fc(final_hidden_state) # Çıktı şekli: (batch_size, output_size)

        return logits

print("\nModel Mimarisi:")
print(model)


Model Mimarisi:
SentimentRNN(
  (embedding): Embedding(20, 32)
  (rnn): RNN(32, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)


## EĞİTİM

In [20]:
# Bu hücre, Jupyter Notebook'unuzda eğitimi başlattığınız hücre olacak.
# Bu hücreyi her çalıştırdığınızda model baştan başlatılacaktır.

# Model Hiperparametreleri (Bu hücrede tanımlanabilir veya önceki hücrelerden alınabilir)
embedding_dim = 32
hidden_size = 32
output_size = 1 # Binary classification

# Eğitim Hiperparametreleri (Bu hücrede tanımlanabilir)
epochs = 500
batch_size = 8
learning_rate = 0.005

# Modeli Yeniden Başlat (Her hücre çalıştırıldığında yeni bir instance oluşturulur)
# Bu adım, modelin ağırlıklarının her zaman rastgele başlangıç değerleriyle başlamasını sağlar.
model = SentimentRNN(vocab_size, embedding_dim, hidden_size, output_size).to(device)
print("Model başarıyla yeniden başlatıldı.")

# Kayıp Fonksiyonu ve Optimizer'ı Yeniden Başlat
# Optimizer, model parametrelerine bağlı olduğu için modelle birlikte yeniden başlatılmalıdır.
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
print("Kayıp fonksiyonu ve Optimizer yeniden başlatıldı.")

# Veri Yükleyicilerini Yeniden Oluştur (Batch boyutu veya shuffle değişirse)
# Eğer batch_size bu hücrede tanımlandıysa, DataLoader'ları burada oluşturmak mantıklı.
from torch.utils.data import TensorDataset, DataLoader

# Dikkat: train_indices_tensor ve train_labels_tensor'ın önceki hücrelerde oluşturulduğunu varsayıyoruz.
train_dataset = TensorDataset(train_indices_tensor, train_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Dikkat: test_indices_tensor ve test_labels_tensor'ın önceki hücrelerde oluşturulduğunu varsayıyoruz.
test_dataset = TensorDataset(test_indices_tensor, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

print("\nEğitim Başlıyor...")
# Eğitim döngüsü
for epoch in range(epochs):
    model.train() # Modeli eğitim moduna al
    total_loss = 0
    correct_predictions = 0
    total_examples = 0

    for batch_indices, batch_labels in train_loader:
        batch_indices, batch_labels = batch_indices.to(device), batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_indices)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch_indices.size(0)
        total_examples += batch_indices.size(0)
        predicted_labels = (torch.sigmoid(outputs) > 0.5).float()
        correct_predictions += (predicted_labels == batch_labels).sum().item()

    avg_train_loss = total_loss / total_examples
    train_accuracy = correct_predictions / total_examples

    # Test (Doğrulama) Seti Üzerinde Değerlendirme
    model.eval() # Modeli değerlendirme moduna al
    total_test_loss = 0
    correct_test_predictions = 0
    total_test_examples = 0

    with torch.no_grad():
        for test_batch_indices, test_batch_labels in test_loader:
            test_batch_indices, test_batch_labels = test_batch_indices.to(device), test_batch_labels.to(device)
            test_outputs = model(test_batch_indices)
            test_loss = criterion(test_outputs, test_batch_labels)

            total_test_loss += test_loss.item() * test_batch_indices.size(0)
            total_test_examples += test_batch_indices.size(0)

            predicted_test_labels = (torch.sigmoid(test_outputs) > 0.5).float()
            correct_test_predictions += (predicted_test_labels == test_batch_labels).sum().item()

    avg_test_loss = total_test_loss / total_test_examples
    test_accuracy = correct_test_predictions / total_test_examples

    if (epoch + 1) % 100 == 0 or epoch == 0: # Daha az çıktı almak için 100 epokta bir yazdır
        print(f"Epoch {epoch+1}/{epochs} | "
              f"Eğitim Kaybı: {avg_train_loss:.4f}, Eğitim Doğruluğu: {train_accuracy:.4f} | "
              f"Test Kaybı: {avg_test_loss:.4f}, Test Doğruluğu: {test_accuracy:.4f}")

print("Eğitim Tamamlandı.")

# Eğitim sonrası test verisi üzerinde nihai değerlendirme veya tahminler
# ... (İsteğe bağlı, önceki koddan alabilirsiniz)

Model başarıyla yeniden başlatıldı.
Kayıp fonksiyonu ve Optimizer yeniden başlatıldı.

Eğitim Başlıyor...
Epoch 1/500 | Eğitim Kaybı: 0.7137, Eğitim Doğruluğu: 0.4483 | Test Kaybı: 0.6974, Test Doğruluğu: 0.5000
Epoch 100/500 | Eğitim Kaybı: 0.3476, Eğitim Doğruluğu: 0.8621 | Test Kaybı: 0.2870, Test Doğruluğu: 0.9000
Epoch 200/500 | Eğitim Kaybı: 0.0028, Eğitim Doğruluğu: 1.0000 | Test Kaybı: 0.0027, Test Doğruluğu: 1.0000
Epoch 300/500 | Eğitim Kaybı: 0.0007, Eğitim Doğruluğu: 1.0000 | Test Kaybı: 0.0007, Test Doğruluğu: 1.0000
Epoch 400/500 | Eğitim Kaybı: 0.0003, Eğitim Doğruluğu: 1.0000 | Test Kaybı: 0.0003, Test Doğruluğu: 1.0000
Epoch 500/500 | Eğitim Kaybı: 0.0002, Eğitim Doğruluğu: 1.0000 | Test Kaybı: 0.0002, Test Doğruluğu: 1.0000
Eğitim Tamamlandı.


## TEST

In [21]:
print("\nTest verisi üzerinde nihai tahminler:")
model.eval() # Modeli değerlendirme moduna al
correct_final_test_predictions = 0

with torch.no_grad(): # Gradyan hesaplamayı kapat
    for sentence, true_label in test_data.items():
        # Tek bir cümle için tahmin yapmak için tensoru hazırla
        indices = sentence_to_indices(sentence, word_to_idx, max_sequence_length)
        input_tensor = torch.LongTensor([indices]).to(device) # Batch boyutu 1 olan tensor

        output = model(input_tensor)
        predicted_prob = torch.sigmoid(output).item() # Logit'ten olasılığa (.item() ile PyTorch tensorundan Python sayısına)
        predicted_label = predicted_prob > 0.5
        true_label_bool = bool(true_label) # Boolean olarak al

        print(f"Cümle: '{sentence}'")
        print(f"  Tahmini Olasılık: {predicted_prob:.4f}")
        print(f"  Tahmin Edilen Etiket: {'Pozitif' if predicted_label else 'Negatif'} (Gerçek: {'Pozitif' if true_label_bool else 'Negatif'})")

        if predicted_label == true_label_bool:
            correct_final_test_predictions += 1

final_test_accuracy = correct_final_test_predictions / len(test_data)
print(f"\nNihai Test Doğruluğu: {final_test_accuracy:.4f}")


Test verisi üzerinde nihai tahminler:
Cümle: 'this is happy'
  Tahmini Olasılık: 0.9999
  Tahmin Edilen Etiket: Pozitif (Gerçek: Pozitif)
Cümle: 'i am good'
  Tahmini Olasılık: 0.9999
  Tahmin Edilen Etiket: Pozitif (Gerçek: Pozitif)
Cümle: 'this is not happy'
  Tahmini Olasılık: 0.0002
  Tahmin Edilen Etiket: Negatif (Gerçek: Negatif)
Cümle: 'i am not good'
  Tahmini Olasılık: 0.0002
  Tahmin Edilen Etiket: Negatif (Gerçek: Negatif)
Cümle: 'this is not bad'
  Tahmini Olasılık: 0.9999
  Tahmin Edilen Etiket: Pozitif (Gerçek: Pozitif)
Cümle: 'i am not sad'
  Tahmini Olasılık: 0.9999
  Tahmin Edilen Etiket: Pozitif (Gerçek: Pozitif)
Cümle: 'i am very good'
  Tahmini Olasılık: 0.9999
  Tahmin Edilen Etiket: Pozitif (Gerçek: Pozitif)
Cümle: 'this is very bad'
  Tahmini Olasılık: 0.0002
  Tahmin Edilen Etiket: Negatif (Gerçek: Negatif)
Cümle: 'i am very sad'
  Tahmini Olasılık: 0.0002
  Tahmin Edilen Etiket: Negatif (Gerçek: Negatif)
Cümle: 'this is bad not good'
  Tahmini Olasılık: 0.0002